# Optimizing the Gitcoin Grants for the Optimality Gap

In [1]:
%load_ext autotime
import matplotlib.pyplot as plt
from collections import defaultdict
from model.parts.utils import *
import networkx as nx
import netwulf as nw
from networkx import community
import plotly.express as px
import numpy as np
import pandas as pd
import cloudpickle
import inequality_coefficients as ineq
from env_config import PICKLE_PATH
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

import plotly.io as pio

time: 2.53 s (started: 2021-01-19 15:54:33 -03:00)


In [2]:
# This used to make the experiment reproducible.
# Change it if you want to have different random numbers
SEED = 5
pio.renderers.default = "jupyterlab"

# Default visualization parameters for the network plots
NETWORK_VIZ_CONFIG= {'node_fill_color': '#79aaa0',
                     'node_size': 25,
                     'zoom': 0.7,
                     'gravity': 0.12,
                     'charge': -39,
                     'link_width': 1.1
                    }

time: 92.5 ms (started: 2021-01-19 15:54:36 -03:00)


In [3]:
# Load the data generated by the cadCAD simulation
with open(PICKLE_PATH, 'rb') as fid:
    result = cloudpickle.load(fid)

time: 21.6 ms (started: 2021-01-19 15:54:36 -03:00)


In [4]:
# Get the last graph
last_state = result.iloc[-1]

# Sum all the contributions for each contributor-grant pair
contributions_totals = (pd.DataFrame(last_state.contributions)
                          .groupby(['contributor', 'grant'])
                          .amount
                          .sum()
                          .pipe(pd.DataFrame)
                          )

time: 10.6 ms (started: 2021-01-19 15:54:36 -03:00)


In [5]:
def total_amount(G: nx.Graph,
                 node: str) -> float:
    """
    Get the total amount in USDT of contribution
    for a given node (grant or contributor)
    """
    # Get all the graph edges associated with the node
    node_edges = G.edges([node])
    
    # Sum all amounts contained in the edges that contains the node
    total_amount = sum(G.edges[edge]['amount']
                       for edge
                       in node_edges)
    
    # Return it
    return total_amount

def contributions_to_graph(contrib_row: list) -> nx.Graph:
    """
    Convert a contributions row from the cadCAD results DataFrame
    into a NetworkX graph.
    """
    # Make sure that we have data
    if len(contrib_row) > 0:
        
        # Load data
        c_df = pd.DataFrame(contrib_row)
        
        # Parse the contributions into a NetworkX graph
        G = nx.from_pandas_edgelist(c_df,
                                    source='contributor',
                                    target='grant',
                                    edge_attr=True)
        
        # Get unique grants and contributors
        unique_grants = c_df.grant.unique()
        unique_contributors = c_df.contributor.unique()
        
        # Associate the 'type' and 'total_amount' attributes for grants and contributors
        grant_node_type = {el: {'type': 'grant',
                                'total_amount': total_amount(G, el)} 
                           for el in unique_grants}
        contrib_node_type = {el: {'type': 'contributor',
                                  'total_amount': total_amount(G, el)} 
                             for el in unique_contributors}
        node_type = {**grant_node_type, **contrib_node_type}    
        
        nx.set_node_attributes(G, node_type)
        
        # Return the graph
        return G
    else:
        return None

# Generate a time series of NetworkX graphs
graphs = result.contributions.map(contributions_to_graph)

time: 612 ms (started: 2021-01-19 15:54:36 -03:00)


In [6]:
G = graphs.iloc[-10]

time: 1.06 ms (started: 2021-01-19 15:54:37 -03:00)


In [7]:
from optimality_gap.subgraph_optimizer import optimize_graph_connectivity
from optimality_gap.quadratic_match import total_quadratic_match

THRESHOLD = 0.3
utility = lambda x: total_quadratic_match(x, THRESHOLD, 'amount')

print(utility(G))
o = optimize_graph_connectivity(G,
                                utility,
                                'grant')

33.800000000000004
Optimizing subgraph



time: 17.9 s (started: 2021-01-19 15:54:37 -03:00)


time: 949 µs (started: 2021-01-19 15:56:01 -03:00)


In [38]:
sG = G.subgraph(list(G.nodes)[70:])
print(optimize_graph_connectivity(sG, utility, 'grant')[1])
print(utility(sG))


Optimizing subgraph



5.199999999999999
5.199999999999999
time: 139 ms (started: 2021-01-19 15:56:46 -03:00)


In [40]:
sG.nodes

NodeView(('bc9b998c2b', 'e7c56c10e0', 'LUKSO: The Blockchain For New Digital Lifestyle', 'Fuzzing @ Home - helping find bugs in ETH 2.0 clients', '07b349e064', 'Rotki - The portfolio tracker and accounting tool that protects your privacy', '0b5a8c6599', 'OrganicBlock - Redefining Urban Farming!', 'POAP (Proof of Attendance Protocol)', 'beaconcha.in - Open source Eth2 blockchain explorer', 'Our Bible', '640dc1af2b', 'ETH2 Staking Guides by CoinCashew', 'Ethereum Staking Guides (Ubuntu)', '23d124901e', 'ef5e2582f1', '8b3d606762', 'ff734df6f7'))

time: 5.14 ms (started: 2021-01-19 15:56:52 -03:00)
